# Exercise 1

Conceptual: Imagine an LLM-based agent that can either answer immediately or invoke a step-by-step reasoning chain (taking more time). Formulate a Meta-RL setup: states might include intermediate reasoning results or uncertainties, actions are “continue reasoning” or “stop and output answer”. What would a reasonable reward be? Perhaps +1 for a correct answer minus a penalty for each step used. Discuss how you would train this (maybe simulate simple math problems of varying difficulty, where continuing reasoning helps for hard ones but is wasteful for easy ones). How is this meta-level decision similar to a bandit? (Hint: each query presents a trade-off between using more resource vs. risk of being wrong, and you have to learn a policy that maps query features to an appropriate compute allocation.)

## Solution

In a meta‑reinforcement learning formulation, the agent’s state would encompass features of the current query and the partial reasoning state, such as the question’s complexity and any intermediate conclusions or uncertainty estimates generated so far. The action space consists of two discrete choices: either continue the chain-of-thought, incurring additional reasoning steps, or stop and output the current best answer. A sensible reward signal would grant a positive reward for producing a correct answer and subtract a cost proportional to the number of reasoning steps used, reflecting the compute budget. To train such a policy, one could simulate tasks of varying difficulty, where shallow reasoning suffices for easy cases but deeper reasoning is required for hard ones. By sampling many tasks, the agent learns to map problem features to appropriate compute allocation decisions, updating its policy via gradient-based reinforcement learning to maximise expected reward. This meta-level choice parallels a bandit problem: each query offers an unknown payoff distribution over compute and correctness, and the agent must learn a strategy that balances the benefit of exploring further reasoning against the risk and cost of being wrong or wasting steps.

The model could also learn not to answer—or to stop early—when a question is too difficult or effectively impossible, so it doesn’t waste tokens (e.g., trying to solve the Navier–Stokes equations).

# Exercise 2

Math/Analysis: In a risk-sensitive objective, say we want to maximize the probability of getting at least one correct solution in 3 tries (like pass@3 for coding). If the model’s per-try success probability is $p$ when optimizing for expected success, it might settle at $p=0.5$ each try (so expected 0.5, and pass@3 ~ 1 - 0.5^3 = 0.875). But a risk-seeking policy might prefer a strategy that yields $p=0.2$ on each try but occasionally a near-perfect attempt, resulting in one attempt being correct with higher probability (this is hypothetical). How would you formalize the reward for pass@3? (One way: reward = 1 if any of the 3 attempts is correct, 0 otherwise.) Why is this a non-linear, non-additive reward that standard RL would struggle with? What algorithms or approaches can handle this (hint: transform it into an auxiliary MDP or use CVaR-based optimization)?

## Solution

A practical way to model **pass@3** is as a binary (Bernoulli) outcome over a *set* of three attempts: you receive reward (1) if **at least one** of the three sampled outputs is correct (e.g., passes unit tests), and reward (0) otherwise. If $f(x_i)\in\{0,1\}$ indicates whether the $i$-th attempt is correct, then the pass@3 objective can be written as

$$
\mathrm{Pass@3}=\mathbb{E}\left[1-\prod_{i=1}^{3}\bigl(1-f(x_i)\bigr)\right],
$$

which is exactly “one minus the probability that all three attempts fail.” This reward is **non-linear and non-additive** because it is an OR over attempts: once any attempt succeeds, extra successes do not increase reward, and you cannot decompose the objective into a sum of per-attempt rewards without changing the problem. As a result, standard RL (built around additive, stepwise returns) suffers from poor credit assignment: the final 0/1 signal depends on the joint outcome of all three samples, so gradients are high-variance and can effectively vanish when successes are rare.

One standard workaround is to re-express the problem as an **auxiliary MDP** by augmenting the *true environment state* so the objective becomes Markov. In practice you should separate (i) the true environment state you use to define the objective from (ii) what the policy is allowed to observe at inference. For pass@3, the true state can be written as

$$
s_t=(t,\text{solved}),
$$

where $t\in\{0,1,2,3\}$ is the attempt index (always known because you control how many samples you have drawn) and $\text{solved}\in\{0,1\}$ means “at least one of the attempts so far is actually correct.” The policy then chooses whether to spend compute to generate another attempt (until $t=3$) or stop; transitions update $t\leftarrow t+1$ and set $\text{solved}=1$ if the new attempt is correct, and the terminal reward is simply $\text{solved}$. This explicit augmentation preserves the Markov property because the future reward depends only on $(t,\text{solved})$, not the full history, which is what standard RL needs.

Crucially, $\text{solved}$ is often **latent** at inference when you cannot run a checker: it exists conceptually to define the correct objective, but you do not get to read it. What you do observe is

$$
o_t,
$$

such as the prompt, the candidate answers generated so far, their logprobs/entropy, self-consistency statistics across samples, and optionally a learned verifier score. Training can still use the latent flag because you can evaluate correctness offline (unit tests, exact match) to compute the pass@3 reward and to label prefixes/attempt-sets; at inference you replace the latent flag with a belief

$$
\hat p_t \approx \Pr(\text{solved}=1\mid o_t)
$$

predicted by a learned success-probability head or a verifier model. The resulting stopping/compute-allocation rule is effectively: stop if $\hat p_t$ is high enough, or if the expected marginal gain from drawing one more attempt (estimated from data as the typical increase in $\hat p$ under similar $o_t$) is smaller than its cost. If you do have a real checker at inference (e.g., unit tests for code), then $\text{solved}$ is no longer latent: you can set $\text{solved}=1$ the moment any attempt passes, making the auxiliary MDP fully observable and straightforward. If you do not have a checker, you are operating in a **POMDP** (Partially Observable MDP): you keep $t$ explicit, keep $\text{solved}$ implicit, and use $\hat p_t$ (or a distribution over outcomes) as the sufficient statistic the policy conditions on. Finally, if a **verifier** is available but expensive (e.g., formal proof checking, compilation + tests, symbolic validation), you can treat “invoke verifier now” as an explicit action with a known cost; this makes the hidden $\text{solved}$ state effectively observable *on demand*, and the optimal policy learns when it is worth paying to reveal correctness versus continuing to sample or reason.


# Exercise 3

Coding (advanced, optional): Given two objectives, e.g., Quality and Latency, implement a simple multi-objective bandit simulation. Arm A gives high quality (reward1 ~ 0.9) at high latency (reward2 ~ -1.0 second), Arm B gives moderate quality (0.7) at low latency (-0.2). Run a bandit algorithm that tries to maximize a weighted sum of these rewards, varying the weight between quality vs. latency. Show how the chosen arm shifts as you change the weight. Then compute the Pareto-optimal set of the two arms (in this case, both might be Pareto-optimal if one is better in quality and the other in latency). Discuss how in a real system we might maintain a set of configurations (prompts/models) that lie on the Pareto frontier of (quality, cost, latency), and possibly choose among them based on real-time user preferences or contexts (for instance, a user on a slow device might implicitly prefer faster responses over slightly higher quality).

## Solution

In [ ]:
# implementation